In [ ]:
import pandas as pd

In [ ]:
!git clone https://github.com/amazon-science/expunations.git

fatal: destination path 'expunations' already exists and is not an empty directory.


In [ ]:
!wget -O semeval2017_task7.tar.xz https://tudatalib.ulb.tu-darmstadt.de/bitstream/handle/tudatalib/2445/semeval2017_task7.tar.xz?sequence=1&isAllowed=y 
!tar -xf semeval2017_task7.tar.xz

--2022-12-09 00:46:57--  https://tudatalib.ulb.tu-darmstadt.de/bitstream/handle/tudatalib/2445/semeval2017_task7.tar.xz?sequence=1
Resolving tudatalib.ulb.tu-darmstadt.de (tudatalib.ulb.tu-darmstadt.de)... 130.83.152.157
Connecting to tudatalib.ulb.tu-darmstadt.de (tudatalib.ulb.tu-darmstadt.de)|130.83.152.157|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 748424 (731K) [application/octet-stream]
Saving to: ‘semeval2017_task7.tar.xz’

semeval2017_task7.t 100%[===================>] 730.88K  1.43MB/s    in 0.5s    

Last-modified header invalid -- time-stamp ignored.
2022-12-09 00:46:58 (1.43 MB/s) - ‘semeval2017_task7.tar.xz’ saved [748424/748424]



In [ ]:
%ll expunations/data/

total 3264
-rw-r--r-- 1 root 3228322 Dec  9 00:46 expunations_annotated_full.json
-rw-r--r-- 1 root  109563 Dec  9 00:46 expunations_annotated_pilot_100.json


In [ ]:
#expun = pd.read_json("expunations/data/expunations_annotated_pilot_100.json").rename(columns={'ID': 'pun_id'}).set_index('pun_id')
expun = pd.read_json("expunations/data/expunations_annotated_full.json").rename(columns={'ID': 'pun_id'}).set_index('pun_id')

#expun.head()

In [ ]:
def get_puns(path):
  import xml.etree.ElementTree as ETree
  xmldata = path
  prstree = ETree.parse(xmldata)
  root = prstree.getroot()

  puns = {}
  i=0
  for sequence in root.iter('text'):
    id = sequence.attrib.get('id')
    pun = []
    for sequence_word in sequence.iter():
      if sequence_word.text != '\n    ':
        pun.append(sequence_word.text)
    puns[i] = {'id': id, 'pun': pun}
    i += 1

  # puns: {'row_0': {'id': 't_1', 'pun': ['When', 'the', ... ]}}
  return pd.DataFrame.from_dict(puns, orient='index')

In [ ]:
heterographic = get_puns("semeval2017_task7/data/test/subtask1-heterographic-test.xml").rename(columns={'id': 'pun_id'}).set_index('pun_id')
homographic = get_puns("semeval2017_task7/data/test/subtask1-homographic-test.xml").rename(columns={'id': 'pun_id'}).set_index('pun_id')
heterograph_labels = pd.read_csv("semeval2017_task7/data/test/subtask1-heterographic-test.gold", names = ['id', 'label'], delim_whitespace=True).rename(columns={'id': 'pun_id'}).set_index('pun_id')
homograph_labels = pd.read_csv("semeval2017_task7/data/test/subtask1-homographic-test.gold", names = ['id', 'label'], delim_whitespace=True).rename(columns={'id': 'pun_id'}).set_index('pun_id')
# The Semeval trial dataset is really only a very very small test dataset.

# Putting it all together

In [ ]:
heterographic['label'] = heterograph_labels['label']
homographic['label'] = homograph_labels['label']
labelled_puns = pd.concat([heterographic, homographic])
pd.reset_option('display.max_colwidth', None)
dataset = expun.join(labelled_puns, on='pun_id', how='inner')
print(dataset.tail())

         Understand the text? Offensive/Inappropriate?       Is a Joke?  \
pun_id                                                                    
hom_2076      [1, 1, 1, 1, 0]           [0, 0, 0, 0, ]  [0, 1, 1, 0, 0]   
hom_1437      [1, 1, 1, 0, 1]           [0, 0, 0, , 0]  [1, 1, 1, 0, 1]   
het_1530      [1, 1, 1, 0, 1]           [0, 0, 0, , 0]  [1, 1, 1, 0, 1]   
het_100       [1, 1, 1, 1, 1]          [0, 0, 0, 0, 0]  [1, 1, 1, 1, 1]   
hom_364       [1, 1, 1, 0, 1]           [0, 0, 0, , 0]  [1, 1, 1, 0, 1]   

          Funniness (1-5)                       Natural language explanation  \
pun_id                                                                         
hom_2076  [0, 1, 1, 0, 0]  [, This is a pun on the phrase 'stroke of geni...   
hom_1437  [1, 1, 2, 0, 1]  [driven can mean to drive a car or it can me f...   
het_1530  [1, 1, 1, 0, 1]  [son sounds like sun , This is a pun on 'sun w...   
het_100   [1, 1, 3, 1, 2]  [lion sounds like line , This is a pun on the .

In [ ]:
del labelled_puns
del heterographic
del homographic 
del homograph_labels

#Trying a different approach here:
- update: it works! kind of. new error in t5 training

In [ ]:
import numpy as np

dataset['Joke keywords'] = dataset['Joke keywords'].apply(np.hstack) #flatten
dataset['Joke keywords'] = dataset['Joke keywords'].apply(str) #bc text input must be of type str
dataset['Joke keywords'] = dataset['Joke keywords'].apply(str.lower) 
dataset['pun'] = dataset['pun'].apply(str) 
dataset['pun'] = dataset['pun'].apply(lambda x: ''.join(x)) 
print("a_process")
print(dataset['Joke keywords'].head())
print(dataset['pun'].head())

a_process
pun_id
hom_362     ['true' 'teeth' 'false' '' 'be true' 'teeth' '...
het_837     ['drinking' 'leave of absinthe' 'drinking too ...
het_635     ['animals' 'soil' 'escape root' 'animals' 'tun...
hom_657     ["waitress's" 'dish it out' '' '' 'waitress' '...
het_1275    ['' 'gossip' 'sense of rumor' 'gossip' 'sense ...
Name: Joke keywords, dtype: object
pun_id
hom_362     ['Be', 'true', 'to', 'your', 'teeth', ',', 'or...
het_837     ['Drinking', 'too', 'much', 'of', 'a', 'certai...
het_635     ['Animals', 'that', 'tunnel', 'in', 'the', 'so...
hom_657     ['She', 'was', 'only', 'a', 'Waitress', "'", '...
het_1275    ['A', 'gossip', 'is', 'someone', 'with', 'a', ...
Name: pun, dtype: object


In [ ]:
#train/test dataset
from sklearn.model_selection import train_test_split
from pandas.core.common import flatten

#need source & target
dataset = dataset[dataset['label'] ==1] #getting only the positive datapoints
limited_dataset = dataset[['Joke keywords', 'pun']] #only taking the columns we need

#need to rename columns for t5 training
limited_dataset.columns = ["source_text", "target_text"]
#prepend the prefix:
limited_dataset['source_text'] = "generate sentence: " + limited_dataset['source_text']

print(limited_dataset.head())

train, test = train_test_split(limited_dataset, test_size=0.3, random_state=0)
#split in half to get both test and eval:
test, eval = train_test_split(test, test_size=0.5, random_state=0)


                                                source_text  \
pun_id                                                        
hom_362   generate sentence: ['true' 'teeth' 'false' '' ...   
het_837   generate sentence: ['drinking' 'leave of absin...   
het_635   generate sentence: ['animals' 'soil' 'escape r...   
hom_657   generate sentence: ["waitress's" 'dish it out'...   
het_1275  generate sentence: ['' 'gossip' 'sense of rumo...   

                                                target_text  
pun_id                                                       
hom_362   ['Be', 'true', 'to', 'your', 'teeth', ',', 'or...  
het_837   ['Drinking', 'too', 'much', 'of', 'a', 'certai...  
het_635   ['Animals', 'that', 'tunnel', 'in', 'the', 'so...  
hom_657   ['She', 'was', 'only', 'a', 'Waitress', "'", '...  
het_1275  ['A', 'gossip', 'is', 'someone', 'with', 'a', ...  


<ipython-input-22-2e9c560c3cc8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  limited_dataset['source_text'] = "generate sentence: " + limited_dataset['source_text']


In [ ]:

!pip install simplet5


In [ ]:
# Training T5

from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained("t5","t5-base")

# train (from simplet5 docs)
# but first we need the data in the right format! see block above

'''model.train(train_df=train, # pandas dataframe with 2 columns: source_text & target_text
            eval_df=eval, # pandas dataframe with 2 columns: source_text & target_text
            source_max_token_len = 512, 
            target_max_token_len = 128,
            batch_size = 8,
            max_epochs = 5,
            use_gpu = False,
            outputdir = "outputs",
            early_stopping_patience_epochs = 0,
            precision = 32
            )'''

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

'model.train(train_df=train, # pandas dataframe with 2 columns: source_text & target_text\n            eval_df=eval, # pandas dataframe with 2 columns: source_text & target_text\n            source_max_token_len = 512, \n            target_max_token_len = 128,\n            batch_size = 8,\n            max_epochs = 5,\n            use_gpu = False,\n            outputdir = "outputs",\n            early_stopping_patience_epochs = 0,\n            precision = 32\n            )'

In [ ]:
#get the trained model data

from google.colab import drive
drive.mount('/content/drive')
_path = "/drive/MyDrive/Colab Notebooks/"

%cd "drive/MyDrive/Colab Notebooks/"

!tar -xf simplet5-epoch-4-train-loss-0.3246-val-loss-0.3666.tar.gz

In [ ]:
# import model
#model.load_model("t5",path+"simplet5-epoch-5-train-loss-1.4723", use_gpu=False)

model.load_model("t5",_path+"simplet5-epoch-4-train-loss-0.3246-val-loss-0.3666", use_gpu=False)

In [ ]:
# run it on the test data
for i in test.index:
        candidateSentences1 = model.predict(test.at[i, "source_text"])
        print(test.at[i, 'source_text'])
        print("Candidate sentences : ", candidateSentences1)

generate sentence: ['' '' 'climb' 'incline' 'ascent' 'yes' 'incline' 'ascent' 'yes' 'climb'
 'steep' 'incline']
Candidate sentences :  ['[\'If\', \'you\', \'can\', \'climb\', \'a\',\'steep\', \',\', \'it\', "\'",\'s\', \'an\', \'incline\', \'.\']']
generate sentence: ['cost' 'space program' 'astronomical' 'cost' 'space program'
 'astronomical' 'space program' 'space' 'astronomical' 'cost'
 'space program' 'astronomical']
Candidate sentences :  ["['The', 'cost', 'of', 'the','space', 'program', 'is', 'astronomical', '.']"]
generate sentence: ['' 'swallowed' 'broken window' 'painfully' '' 'swallowed' 'glass'
 'broken window' '']
Candidate sentences :  ['["\'", "\'", \'I\',\'swallowed\', \'the\', \'glass\', \'in\', \'a\', \'broken\', \'window\', \'painfully\', \'.\']']
generate sentence: ['' 'guitarists' 'die' 'fret' 'guitarists' 'fret' 'old guitarists' 'die'
 'fret their lives away' 'guitarists' 'die' 'fret their lives away']
Candidate sentences :  ["['OLD', 'GUITARISTS', 'never', 'die', 